# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
* Usage of Strings
* Method Signatures (Modifiers, Parameter Types, Number of Parameters...)
* Other methods in same class
* Analyze Method Calls (from and to) via Call Graph (Distance, Offsets, Graph Analysis)

In [1]:
%matplotlib notebook

In [2]:
import unittest
from collections import defaultdict, Counter
from os import path

from androguard.core.analysis.analysis import MethodAnalysis, ClassAnalysis, FieldAnalysis
from androguard.core.bytecode import FormatClassToJava
from androguard.misc import AnalyzeAPK
from androguard.session import Save, Session, Load

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MAX_USAGE_COUNT_STR = 20
UNIQUE_STRINGS_MAJORITY = 2 / 3
MULTIPROCESS_FILES = True

file_paths = (
    "../../../Downloads/com.snapchat.android_10.85.5.74-2067_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)

In [4]:
def load_androguard(file_path, force_reload=False, write_session=True):
    # Writing and Loading sessions currently cause a Kernel Disconnect or an EOF Error
    if (not force_reload) and path.exists(AG_SESSION_FILE):
        print("Loading Existing Session")
        s = Load(AG_SESSION_FILE)
    else:
        print("Loading Session from Apk")
        s = Session()
        a, d, dx = AnalyzeAPK(file_path, s)
        if write_session:
            print("Saving Loaded Session to", AG_SESSION_FILE)
            Save(s, AG_SESSION_FILE)
    return a, d, dx

In [5]:
# Multiprocessing not working, probably same issue regarding serialization than when trying to write/load 
# androguard sessions
"""
def multiprocess_files(file_paths):
    parent_conn, child_conn = multiprocessing.Pipe(False)


    def post_result(file_path, conn):
        value = load_androguard(file_path, True, False)
        conn.send((file_path, value))

    ps =  [multiprocessing.Process(target=post_result, args=(f, child_conn)) for f in file_paths]

    def apply_map(f, i):
        for x in i:
            f(x)
    
    assert len(file_paths) == 2
    print("Starting multiprocessing Files")
    
    # Serialization with Pickle requires higher recursion limit
    import sys
    previous_recursion = sys.getrecursionlimit()
    sys.setrecursionlimit(50000)
    
    
    apply_map(multiprocessing.Process.start, ps)

    values = (q.get(), q.get())
    r = tuple(map(lambda x: x[1], sorted(values, key=lambda x: file_paths.index(x[0]))))
    
    
    apply_map(multiprocessing.Process.join, ps)
    
    print("Finished all processes")
    sys.setrecursionlimit(previous_recursion)
    return r

if MULTIPROCESS_FILES:
    (a, d, dx), (a2, d2, dx2) = multiprocess_files(file_paths)
else:
    (a, d, dx), (a2, d2, dx2) = tuple(map(lambda x: load_androguard(x, True, False), file_paths))
"""
a, d, dx = load_androguard(file_paths[0], True, False)

Loading Session from Apk


### Utility Functions to work with Androguard and Java Representations

* Converting Parameter types to TypeDescriptor Format
* Strip return type (not used for hooking)
* Method Representation Format

Loaded with Unit Tests

In [6]:
# https://source.android.com/devices/tech/dalvik/dex-format#typedescriptor
type_descriptors = {
    "void": "V",
    "boolean": "Z",
    "byte": "B",
    "short": "S",
    "char": "C",
    "int": "I",
    "long": "J",
    "float": "F",
    "double": "D"
}

type_ds_reversed = { v : k for k, v in type_descriptors.items() }

def get_as_type_descriptor(arg):
    if arg.endswith("[]"):
        return "[" + get_as_type_descriptor(arg[:-2])
    if arg in type_descriptors:
        return type_descriptors[arg]
    return FormatClassToJava(arg)

In [7]:
def strip_return_descriptor(descriptor):
    return descriptor[1:descriptor.index(")")]

In [8]:
def pretty_format_class(class_name):
    if class_name.startswith("["):
        return pretty_format_class(class_name[1:]) + "[]"
    if class_name in type_ds_reversed:
        return type_ds_reversed[class_name]
    return class_name[1:-1].replace("/", ".")


def get_pretty_params(descriptor):
    return map(pretty_format_class, strip_return_descriptor(descriptor).split(" "))

In [9]:
def get_method_repr(class_name, method_name, param_types):
    return f"{class_name}#{method_name}({param_types})"

def pretty_format_ma(ma):
    return get_method_repr(pretty_format_class(ma.class_name), ma.name, ", ".join(get_pretty_params(str(ma.descriptor))))

In [10]:
tests_1 = (
    ("java.lang.String", "Ljava/lang/String;"),
    ("java.lang.String[]", "[Ljava/lang/String;"),
    ("void", "V"),
    ("int[]", "[I"),
    ("char", "C"), 
    ("java.lang.Object[][]", "[[Ljava/lang/Object;"),
    ("ABC", "LABC;")
)

tests_2 = (
    ("(I)I", "I"), 
    ("(C)Z", "C"),
    ("(Ljava/lang/CharSequence; I)I", "Ljava/lang/CharSequence; I")
)

class TestFunction(unittest.TestCase):
    def test_type_descriptor(self):
        for test, val in tests_1:
            self.assertEqual(get_as_type_descriptor(test), val)
    
    def test_strip_return(self):
        for test, val in tests_2:
            self.assertEqual(strip_return_descriptor(test), val)
    
    def test_pretty_class(self):
        for val, test in tests_1:
            self.assertEqual(pretty_format_class(test), val)


unittest.main(argv=[''], verbosity=2, exit=False)

test_pretty_class (__main__.TestFunction) ... ok
test_strip_return (__main__.TestFunction) ... ok
test_type_descriptor (__main__.TestFunction) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


# Method Declarations

Lightweight Method Declaration for internal representation of a Method / Hook.

Not keeping Androguard Objects in memory to avoid high memory usage.

In [11]:
class MethodDec:
    def __init__(self, class_name, name, *param_types, skip_params=False):
        self.name = name
        self.class_name = class_name
        self.param_types = param_types
        self.skip_params = skip_params
    
    
    def get_formatted_param_types(self):
        return list(map(get_as_type_descriptor, self.param_types))
    
    
    def param_types_repr(self):
        if self.skip_params:
            return "skip.params"
        return " ".join(self.get_formatted_param_types())
    
    
    def get_formatted_class(self):
        return FormatClassToJava(self.class_name)
    
    
    def pretty_format(self):
        return get_method_repr(self.class_name, self.name, 
                               ("skip.params" if self.skip_params else ", ".join(self.param_types)))
    

    def __repr__(self):
        return f'MethodDec({self.pretty_format()})'
    
    
    def equals_ma(self, ma):
        return self.name == ma.name and (self.skip_params or \
            self.param_types_repr() == strip_return_descriptor(str(ma.get_descriptor())))
    
    def find_ma(self, dx):
        for ma in dx.get_class_analysis(FormatClassToJava(self.class_name)).get_methods():
            if self.equals_ma(ma):
                return ma
        raise Exception(f"Unresolved MethodDec: {self.pretty_format()}")

### List of Methods

Defining the list of methods to find (obviously requires full class names)

In [12]:
decs_to_find = [
    MethodDec("rD5", "a", "rD5", "qD5"),
    MethodDec("MSg", "j0", "SGd"),
    MethodDec("x45", "h"),
    MethodDec("GIb", "<init>", skip_params=True)
]

# Processing

## Strings as Characteristics

Extracting Strings used either in the given methods directly or in the classes the methods define

In [13]:
# Key:   TypeDescriptor Representation of class
# Value: Androguard Class Analysis Object

resolved_classes = { i: dx.get_class_analysis(i)
                    for i in map(lambda x: FormatClassToJava(x.class_name), decs_to_find) }

In [14]:
resolved_methods = [m.find_ma(dx) for m in decs_to_find]

print("Resolved all Classes and Methods", *map(pretty_format_ma,resolved_methods), sep="\n* ")

Resolved all Classes and Methods
* rD5#a(rD5, qD5)
* MSg#j0(SGd)
* x45#h()
* GIb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Ds5, long, boolean, java.lang.Long, long)


### Utility functions for working with dx.get_strings()

Filters Strings and xrefs to Strings. Only allow strings with (#xrefs < MAX_USAGE_COUNT_STR) to be used as characteristic to locate classes

In [15]:
def get_filtered_strs(dx):
    """
    Loops through all strings that are referenced less than MAX_USAGE_COUNT_STR times and hence can be 
    used as characteristic for finding methods or classes.
    """
    return ((s, xrefs) 
            for s, xrefs in map(lambda s: (s, s.get_xref_from()), dx.get_strings()) 
            if len(xrefs) <= MAX_USAGE_COUNT_STR)


def get_xrefs_if_usable(s):
    """
    Loops through xrefs of a string only if the number of references does not exceed MAX_USAGE_COUNT_STR.
    """
    xrefs = s.get_xref_from()
    if len(xrefs) > MAX_USAGE_COUNT_STR:
        return
    yield from xrefs

Building Maps of MethodDec and ClassNames associated to lists containing strings used in them

In [16]:
m_strs, c_strs = defaultdict(list), defaultdict(list)

for s, xrefs in get_filtered_strs(dx):
    for x in xrefs:
        c_ref, m_ref = x

        if c_ref.name not in resolved_classes:
            # XReference not in a Class or method that we need to find
            continue

        # Loop through each method and find methods in this class
        for r_m, m_dec in zip(resolved_methods, decs_to_find):
            if r_m.class_name != c_ref.name:
                continue

            # String is used in a class we need to find
            c_strs[c_ref.name].append(s.value)
            
            if m_ref == r_m:
                # String is used in this method
                m_strs[m_dec].append(s.value)

### Count occurrences of strings

Converting list of strings to a Counter object for faster comparisons

In [17]:
m_strs = {k: Counter(v) for k, v in m_strs.items()}
c_strs = {k: Counter(v) for k, v in c_strs.items()}

# If no strings have been found in method, still insert a Counter of 0
for m in decs_to_find:
    if m not in m_strs:
        m_strs[m] = Counter()

In [18]:
def flat_map(f, li):
    """
    Maps values with function f recursively on all Iterables (except Strings)
    Flattened by using recursive Subgenerator Delegation
    """
    from collections.abc import Iterable
    for i in li:
        # str will cause a recursion depth error (Iterator of str returns Iterable str)
        if isinstance(i, Iterable) and not isinstance(i, str):
            yield from flat_map(f, i)
        else:
            yield f(i)

### Searching for Found Strings

Tries to resolve Classes and methods with the strings previously found

* Loading second Apk File
* Find All Strings found previously, build Map of potential matches (ClassName/Method to Counter)
* Filter Potential Matches by comparing both Counter Objects

In [19]:
a2, d2, dx2 = load_androguard(file_paths[1], True, False)

Loading Session from Apk


In [20]:
m_strs2, c_strs2 = defaultdict(list), defaultdict(list)

for s in dx2.get_strings():
    for m_dec, m_set in m_strs.items():
        
        c_name = m_dec.class_name
        try:
            c_set = c_strs[FormatClassToJava(c_name)]
        except KeyError:
            c_set = set()
        
        if s.value in m_set:
            for x in get_xrefs_if_usable(s):
                m_ref = x[1]
                m_strs2[m_ref].append(s.value)
        if s.value in c_set:
            for x in get_xrefs_if_usable(s):
                c_ref = x[0]
                c_strs2[c_ref.name].append(s.value)

In [21]:
m_strs2 = {k: Counter(v) for k, v in m_strs2.items()}
c_strs2 = {k: Counter(v) for k, v in c_strs2.items()}

#### Try to resolve Method
Try to resolve classes by only using information about Strings (exact Counter Match)

In [22]:
matching_cs, matching_ms = {}, {}
candidates_cs, candidates_ms = defaultdict(set), defaultdict(set)

In [23]:
# Accumulate all candidates with exact same Counter Object for methods and classes
for k2, c2 in c_strs2.items():
    for k1, c1 in c_strs.items():
        if c1 == c2:
            candidates_cs[k1].add(k2)

for k2, c2 in m_strs2.items():
    for k1, c1 in m_strs.items():
        if c1 == c2:
            candidates_ms[k1].add(k2)
            
            
# Filter all candidate items for both classes and methods. Accepting single match as "matching"
for k1, cs_li in candidates_cs.items():
    cs_li = list(cs_li)
    if len(cs_li) == 1:
        print("+ Found Single match for class. Considering a match!"
              + f"\n\t{pretty_format_class(k1)} -> {pretty_format_class(cs_li[0])}")
        matching_cs[k1] = cs_li[0]
        continue
        
    print(f"* Multiple Matches for class {pretty_format_class(k1)}", *map(pretty_format_class, cs_li), sep="\n\t* ")

for k1, ms_li in candidates_ms.items():
    ms_li = list(ms_li)
    if len(ms_li) == 1:
        print("+ Found Single match for method. Considering a match!"
              + f"\n\t{k1.pretty_format()} -> {pretty_format_ma(ms_li[0])}")
        matching_ms[k1] = ms_li[0]
        continue
        
    print(f"* Multiple Matches for method {k1.pretty_format()}", *map(pretty_format_ma, ms_li), sep="\n\t* ")

    
    
# Delete "matching" from candidates
for mcs in matching_cs.keys():
    del candidates_cs[mcs]
for mms in matching_ms.keys():
    del candidates_ms[mms]

+ Found Single match for class. Considering a match!
	rD5 -> QE5
+ Found Single match for class. Considering a match!
	MSg -> V0h
+ Found Single match for method. Considering a match!
	rD5#a(rD5, qD5) -> QE5#a(QE5, PE5)
+ Found Single match for method. Considering a match!
	MSg#j0(SGd) -> V0h#j0(IMd)


In [24]:
c_not_found = set(map(lambda x: x.class_name, decs_to_find)) - set(map(pretty_format_class, matching_cs.keys()))
m_not_found = decs_to_find - matching_ms.keys()

### Fallback by unique strings

To resolve unresolved methods, get all unique strings (strings only used by this class) and try to find the matching class by only searching for the unique string.

In [25]:
def used_in_single_class(xrefs, cn):
    # if string only used in one class, regardless of the number of references
    return all((xref[0].name == cn for xref in xrefs))


unique_strs = defaultdict(set)
for s, xrefs in get_filtered_strs(dx):
    cn = list(xrefs)[0][0].name
    if used_in_single_class(xrefs, cn):
        pf = pretty_format_class(cn)
        if pf in c_not_found:
            unique_strs[pf].add(s.value)


matching_us = defaultdict(set)
strs = set().union(*unique_strs.values())
for s, xrefs in get_filtered_strs(dx2):
    if s.value in strs:
        cn = list(xrefs)[0][0].name
        if used_in_single_class(xrefs, cn):
            matching_us[pretty_format_class(cn)].add(s.value)
        else:
            print(f"~ Unique String not used in single class anymore. Change! ({s.value})")


tmp_candidates_cs = defaultdict(list)
for c1, strset1 in unique_strs.items():
    for s1 in strset1:
        for c2, strset2 in matching_us.items():
            for s2 in strset2:
                if s1 == s2:
                    tmp_candidates_cs[str(c1)].append(str(c2))

tmp_candidates_cs = { k: Counter(v) for (k, v) in tmp_candidates_cs.items() }
for c, counter in tmp_candidates_cs.items():
    if len(counter) == 1:
        c_name, count = tuple(*counter.items())
        print("+ Found single class candidate via Unique Strings"
             + f"\n\t{c} -> {c_name}")
        matching_cs[FormatClassToJava(c)] = FormatClassToJava(c_name)
        continue
    
    
    total = sum(counter.values())
    print(f"* Found multiple classes for all unique strings. ({len(counter)} candidates)")
    for c2, count in counter.items():
        if count / total >= UNIQUE_STRINGS_MAJORITY:
            print(f"+ Found majority for unique string. Considering a match! ({(count/total):.2f}% majority)"
                  + f"\n\t{c} -> {c2}")
            matching_cs[FormatClassToJava(c)] = FormatClassToJava(c2)
            break

for c, c2 in matching_cs.items():
    if c in tmp_candidates_cs:
        del tmp_candidates_cs[c]
    if c in candidates_cs:
        del candidates_cs[c]

### Using Class Information

Gathers the following information and tries to find the correct classes by finding a similar "Profile"
* Modifiers for Methods and Fields
* "static" Field and return types of Methods
* #Fields and #Methods

In [26]:
def get_usable(class_name):
    if class_name.startswith("["):
        return "[" + get_usable(class_name[1:])
    
    if class_name.startswith("Ljava/") or class_name.startswith("Landroid/") or class_name in type_descriptors.values():
        return class_name
    return "obfuscated.class"


def get_field_counter(ca):
    li = []
    for f in map(FieldAnalysis.get_field, ca.get_fields()):
        if ca.name != f.get_class_name():
            continue
        
        li.append(get_usable(str(f.get_descriptor())))
    return Counter(li)

def get_usable_description(ma):
    stripped, r = str(ma.descriptor[1:]).split(")")
    return "(" + (" ".join(map(get_usable, stripped.split(" "))) if stripped else "") + ")" + get_usable(r)

def get_method_set(ca):
    return { get_usable_description(m) for m in ca.get_methods() }

In [27]:
criteria = [
    ClassAnalysis.get_nb_methods,
    lambda ca: len(list(filter(lambda x: x.get_field().get_class_name() == ca.name, ca.get_fields()))),
    lambda ca: str(get_field_counter(ca)),
    get_method_set
]

tmp_candidates = defaultdict(set)
for c in c_not_found:
    if c.startswith("com.") and dx2.get_class_analysis(FormatClassToJava(c)) is not None:
        c_name = FormatClassToJava(c)
        matching_cs[c_name] = c_name
        print("+ Found Matching Class (by similar name). Accepting immediate match!"
              + f"\n\t{c_name} -> {c_name}")
        continue
        
    ca = dx.get_class_analysis(FormatClassToJava(c))
    for ca2 in dx2.get_classes():
        for i, cr in enumerate(criteria):
            if cr(ca) != cr(ca2):
                break
        else:
            tmp_candidates[ca.name].add(ca2.name)
            print("+ Found Matching Class (by matching critera)"
                  + f"\n\t{c} -> {ca2.name}")


# Filter all candidate items for both classes and methods. Accepting single match as "matching"
for k1, cs_li in tmp_candidates.items():
    cs_li = list(cs_li)
    if len(cs_li) == 1:
        print("+ Found Single match for class. Considering a match!"
              + f"\n\t{pretty_format_class(k1)} -> {pretty_format_class(cs_li[0])}")
        matching_cs[k1] = cs_li[0]
        continue
        
    print(f"* Multiple Matches for class {pretty_format_class(k1)}", *map(pretty_format_class, cs_li), sep="\n\t* ")

    
# Delete "matching" from candidates
for mcs in matching_cs:
    if mcs in candidates_cs:
        del candidates_cs[mcs]
    if mcs in tmp_candidates:
        del tmp_candidates[mcs]
        
# Use inner Join of previous candidates and current candidates to find a single match
print(candidates_cs, tmp_candidates)

+ Found Matching Class (by matching critera)
	GIb -> LFOb;
+ Found Single match for class. Considering a match!
	GIb -> FOb
defaultdict(<class 'set'>, {}) defaultdict(<class 'set'>, {})


In [28]:
c_not_found = set(map(lambda x: x.class_name, decs_to_find)) - set(map(pretty_format_class, matching_cs.keys()))

### Fallback if Class was found

In case the class was found, but the method could not be resolved, check each method of the class for the following criteria:

* Matching #xrefs_to
* Matching #xrefs_from
* Matching Code length

All of these checks are currently strict/exact

In [29]:
# Compare Function
cfs = [
    (MethodAnalysis.get_access_flags_string, 4),
    (get_usable_description, 10),
    (MethodAnalysis.get_length, 1),
    (lambda x: len(x.get_xref_to()), 1),
    (lambda x: len(x.get_xref_from()), 1)
]
total_score = sum((score for _, score in cfs))

In [35]:
MIN_MATCH_POINTS = 2

def try_resolve_ms(exact):    
    candidates_ms2 = defaultdict(set)

    for m in m_not_found:
        if m.class_name in c_not_found:
            print("> Could not find class of method", m.pretty_format())
            continue

        class_name1 = FormatClassToJava(m.class_name)
        class_name2 = matching_cs[class_name1]

        for ma1 in dx.get_class_analysis(class_name1).get_methods():
            if not m.equals_ma(ma1):
                continue

            m_match_points = {}
            for ma2 in dx2.get_class_analysis(class_name2).get_methods():
                
                if exact:
                    if all((c_fun(ma1) == c_fun(ma2)) * score for c_fun, score in cfs):
                        candidates_ms2[m].add(ma2)
                else:
                    x = sum(((c_fun(ma1) == c_fun(ma2)) * score for c_fun, score in cfs))
                    if x >= MIN_MATCH_POINTS:
                        m_match_points[ma2] = x

            if not exact:
                max_matches = max(map(lambda x: x[1], m_match_points.items()))
                c = [s[0] for s in m_match_points.items() if s[1] == max_matches]
                
                if len(c) == 0:
                    print("- Could not find any matching candidate for", pretty_format_ma(ma1))
                elif len(c) == 1:
                    print(f"+ Found single non-exact candidate for matching method. (Certainty of {(max_matches / total_score):.2f})", pretty_format_ma(ma1), pretty_format_ma(c[0]), sep="\n\t* ")
                    matching_ms[m] = c[0]
                else:
                    candidates_ms2[m] |= set(c)
                    print("* Found multiple non-exact candidates for matching method " + str(m) + ":\n\t*", "\n\t* ".join(map(pretty_format_ma, c)))
            break

    for m, ms_li in candidates_ms2.items():
        ms_li = list(ms_li)
        if len(ms_li) == 1:
            print("+ Found single candidate for matching method. Considering it a match!",
                  f"\n\t{m.pretty_format()} -> {ms_li[0]}")

            matching_ms[m] = ms_li[0]
            continue

        print(f"* Multiple Matches for method {m.pretty_format()}", *map(pretty_format_ma, ms_li), sep="\n\t* ")
        
        if m in candidates_ms:
            combined = set(ms_li) & set(candidates_ms[m])
            if len(combined) == 0:
                print("- Inner join on possible candidates resulted in no method match! for", m)
                continue
            if len(combined) == 1:
                el = list(combined)[0]
                print("+ Inner join concluded single matching candidate. Considering match!"
                      + f"\n\t{m.pretty_format()} -> {pretty_format_ma(el)}")
                matching_ms[m] = el
                continue
            if len(combined) < len(candidates[ms]):
                print(f".. Could narrow down search by combining candidates ({len(candidates[ms])} -> {len(combined)})")
                candidates[ms] = combined
            
    for m, ma2 in matching_ms.items():
        if m in candidates_ms:
            del candidates_ms[m]
        
try_resolve_ms(exact=True)
m_not_found = decs_to_find - matching_ms.keys()

print("Using non-exact Checks")

try_resolve_ms(exact=False)
m_not_found = decs_to_find - matching_ms.keys()

print(len(m_not_found), "/", len(decs_to_find))

> Could not find class of method x45#h()
Using non-exact Checks
> Could not find class of method x45#h()
1 / 4


In [31]:
m_not_found = decs_to_find - matching_ms.keys()
print(len(m_not_found), "/", len(decs_to_find))

print()
print("Classes that have not been found:", *c_not_found, sep="\n* ")
print()
print("Resolved MethodDecs:")
for m, ma2 in matching_ms.items():
    print("*", m.pretty_format(), "->", pretty_format_ma(ma2))
for m in m_not_found:
    print("-", m.pretty_format())

1 / 4

Classes that have not been found:
* x45

Resolved MethodDecs:
* rD5#a(rD5, qD5) -> QE5#a(QE5, PE5)
* MSg#j0(SGd) -> V0h#j0(IMd)
* GIb#<init>(skip.params) -> FOb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Xt5, long, boolean, java.lang.Long, long)
- x45#h()


In [32]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))